The threading library can be used to execute any Python callable in 
its own thread. To do this, you create a Thread instance and supply 
the callable that you wish to execute as a target. Here is a simple example:

In [3]:
# code to execute in an independent thread
import time
def countdown(n):
    while n>0:
        print('T-minus', n)
        n -= 1
        time.sleep(5)

In [9]:
# create and launch a thread
from threading import Thread
t = Thread(target=countdown, args=(10,))
t.start()

T-minus 10


Threads are executed in their own system-level 
thread (e.g., a POSIX thread or Windows threads)
that is fully managed by the host operating system. 
Once started, threads run independently until the 
target function returns. You can query a thread 
instance to see if it’s still running:

In [11]:
if t.is_alive():
    print('Still running')
else:
    print('Completed')

Still running
T-minus 9
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4
T-minus 3
T-minus 2


You can also request to join with a thread, which waits 
for it to terminate:

In [13]:
t.join()

The interpreter remains running until all threads terminate. 
For long-running threads or background tasks that run forever, 
you should consider making the thread daemonic. For example:

In [15]:
t = Thread(target=countdown, args=(10,), daemon=True)
t.start()

T-minus 10
T-minus 9
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4


In [16]:
# Daemonic threads can’t be joined. However, they are destroyed 
# automatically when the main thread terminates.

T-minus 3
T-minus 2
T-minus 1


If you want to be able to terminate threads, the thread must be 
programmed to poll for exit at selected points. For example, you
might put your thread in a class such as this:

In [20]:
class CountdownTask:
    def __init__(self):
        self._running = True
    def terminate(self):
        self._running = False
    def run(self, n):
        while self._running and n>0:
            print('T-minus', n)
            n -= 1
            time.sleep(5)

In [21]:
c = CountdownTask()
t = Thread(target=c.run, args=(10,))
t.start()
c.terminate()
t.join()

T-minus 10


Polling for thread termination can be tricky to coordinate 
if threads perform blocking operations such as I/O. For 
example, a thread blocked indefinitely on an I/O operation
may never return to check if it’s been killed. To correctly 
deal with this case, you’ll need to carefully program thread 
to utilize timeout loops. For example:

In [22]:
class IOTask:
    def terminate(self):
        self._running = False
    def run(self, sock):
        # sock is a socket
        sock.settimeout(5)
        while self._running:
            # Perform a blocking I/O operation w/ timeout
            try:
                data = sock.recv(8192)
                break
            except socket.timeout:
                continue
            # Continued processing
        # Terminated
        return

Due to a global interpreter lock (GIL), Python threads 
are restricted to an execution model that only allows 
one thread to execute in the interpreter at any given time. 
For this reason, Python threads should generally not be 
used for computationally intensive tasks where you are trying 
to achieve parallelism on multiple CPUs. They are much better 
suited for I/O handling and handling concurrent execution in 
code that performs blocking operations (e.g., waiting for I/O, 
waiting for results from a database, etc.).

Sometimes you will see threads defined via inheritance 
from the Thread class. For example:

In [26]:
from threading import Thread
class CountdownThread(Thread):
    def __init__(self, n):
        super().__init__()
        self.n = 0
    def run(self):
        while self.n > 0:
            print('T-minus', self.n)
            self.n -= 1
            time.sleep(5)

In [27]:
c = CountdownThread(5)
c.start()

In [28]:
import multiprocessing
c = CountdownThread(5)
p = multiprocessing.Process(target = c.run)
p.start()

Here is some sample code that uses an Event to coordinate the startup of a thread:

In [30]:
from threading import Thread, Event
import time
#Code to execute in an independent thread
def countdown(n, started_evt):
    print('countdown starting')
    started_evt.set()
    while n > 0:
        print('T-minus', n)
        n -= 1
        time.sleep(5)

In [31]:
# Create the event object that will be used to signal startup
started_evt = Event()
# Launch the thread and pass the startup event
print('Launching countdown')
t = Thread(target=countdown, args=(10, started_evt))
t.start()
# Wait for the thread to start
started_evt.wait()
print('countdown is running')

Launching countdown
countdown starting
T-minus 10
countdown is running
T-minus 9
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


## Event Object ##

Event objects are best used for one-time events: when you creates an event, threads wait for the event to be set, and the Event is discarded after that. However usage of event is still error prone, even though the it is possible to clear an event using its clear() method.
If a thread is going to repeatedly signal an event over and over, you're probably better off using a Condition object. For example, a periodic timer:

In [34]:
import threading
import time

class PeriodicTimer:
    def __init__(self, interval):
        self._interval = interval
        self._flag = 0
        self._cv = threading.Condition()
    
    def start(self):
        t = threading.Thread(target=self.run)
        t.daemon = True
        t.start()
    
    def run(self):
        '''
        Run the timer and notify waiting threads after each interval
        '''
        while True:
            time.sleep(self._interval)
            with self._cv:
                self._flag ^= 1
                self._cv.notify_all()
    
    def wait_for_tick(self):
        '''
        Wait for the next tick of the timer
        '''
        with self._cv:
            last_flag = self._flag
            while last_flag == self._flag:
                self._cv.wait()

In [36]:
# Example use of the timer
ptimer = PeriodicTimer(5)
ptimer.start()

In [37]:
# Two threads that synchronize on the timer
def countdown(nticks):
    while nticks > 0:
        ptimer.wait_for_tick()
        print('T-minus', nticks)
        nticks -= 1
        
def countup(last):
    n = 0
    while n < last:
        ptimer.wait_for_tick()
        print('Counting', n)
        n += 1
        
threading.Thread(target=countdown, args=(10,)).start()
threading.Thread(target=countup, args=(5,)).start()

T-minusCounting 10
 0
T-minus 9
Counting 1
T-minus 8
Counting 2
Counting 3
T-minus 7
CountingT-minus 4 6

T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


Event objects wakes all waiting threads. If you are writing a program where you only want to wake up a single waiting thread, it is probably better to use a Semaphore or Condition object instead:

In [38]:
# Worker thread
def worker(n, sema):
    # Wait to be signaled
    sema.acquire()
    # Do some work
    print('Working', n)

# Create some threads
sema = threading.Semaphore(0)
nworkers = 10
for n in range(nworkers):
    t = threading.Thread(target=worker, args=(n,sema,))
    t.start()

In [39]:
sema.release()

Working 0


In [40]:
sema.release()

Working 1


## Communicating Between Threads ##

How to safely communicate or exchange data between them.
## Solution##
safest way o send data between threads is using a queue from queue library: an instance of queue sharee by threads that uses put() or get() operations.

In [ ]:
from queue import Queue
from threading import Thread

# A thread that produces data
def producer(out_q):
    while True:
        #Produce some data
        ...
        for i in range(5):
            out_q.put(i)
# A thread that consumes data
def consumer(in_q):
    while True:
        #Get some data
        data = in_q.get()
        # Process the data
        print( data)
# Create the shared queue and launch both threads
q = Queue()
t1 = Thread(target=consumer, args=(q,))
t2 = Thread(target=producer, args=(q,))
#t1.start()
#t2.start()
#t1.join()
#t2.join()

When using queues, it can be somewhat tricky to coordinate the shutdown of the producer and consumer. A common solution to this problem is to rely on a special sentinel value, which when placed in the queue, causes consumers to terminate. 

In [ ]:
from queue import Queue
from threading import Thread
# Object that signals shutdown
_sentinel = object()
# A thread that produces data
def producer(out_q):
    while running:
        #Produce some data
        ...
        for i in range(5):
            out_q.put(i)
# A thread that consumes data
def consumer(in_q):
    while True:
        #Get some data
        data = in_q.get()
        # Process the data
        if data is _sentinel:
            in_q.put(_sentinel)
            break
# Create the shared queue and launch both threads
q = Queue()
t1 = Thread(target=consumer, args=(q,))
t2 = Thread(target=producer, args=(q,))
t2.start()
t1.start()

Thread communication mechanism can also be implemented with required lock and synchronization. The most common way is conditional variable. Below is a thread-sa

In [ ]:
import headq
import threading

class PriorityQueue
    def __init__(self):
        self._queue = []
        self._count = 0
        self._cv = threading.Condition()
    def put(self, item, priority):
        with self._cv:
            headq.heappush(self._queue, (-priority, self._count, item))
            self._count += 1
            self._cv.notify()
    def get(self):
        with self._cv:
            while len(self._queue) == 0:
                self._cv.wait()
            return heapq.heappop(self._queue)[-1]